# 1 Crime rate and House Prices in Ames City:
In their paper "Crime and Property Values: Evidence from the 1990s Crime Drop” Devin and Jaren Pope bring statistical evidence that crime drop in the 90's in the USA had a postive coraletion on the housing prices in the US. They claim that "a drop in violent crime by 100 per 10,000 people is associated with a 5.5% increase in housing values."*
In this notebook we will try and check of this claim holds for Ames's, Iowa real estate markt in the begining of the 21th centurey. In order to test this conjecture we picked a sample of 4 years out of 18. We used two databases: 
1)Ames real estate sell prices from kaggle: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview
2)The FBI's Uniform Crime Reporting Programm: https://www.fbi.gov/services/cjis/ucr/
To put it to more percise words our null hypothesis is that for the years 2006 till 2010 as a whole for an annual drop of 100 cases of proprety crime for 10,000 residences, we will see increase of around 5.5%. Ergo:         
$$H_0 : S(Amces) = 5.5\%$$

Method:
We will used the our databaces to calculate the annual change in proprerty crime rates and in sales prices drawing the mean of of those years.
As we know the STD of Pope's reachers (8) we can see if can calculate the z score of our result.   


Work flow: 

1:first we lay the fundamntals  

2.importaing databases and calculating the Δs.

3.runing the numbers on the Δs. 

*Pope, Devin G., and Jaren C. Pope. “Crime and Property Values: Evidence from the 1990s Crime Drop.” Regional Science and Urban Economics 42, no. 1–2 (January 2012): 186.

## 1. laying down the fundmantals 

In [321]:
#Libraries Used
import pandas as pd
import numpy as np
import seaborn as sns

###  Basic formulas for Sample statistic
We need some basic formulas to calculate the mean, the standart deviation and the standart score for samples that can take pandas Series   

In [322]:
def mean_sample(ser):
    sum_ser, counter = 0, 0
    for i in list(ser):
        counter+=1
        sum_ser+=i
    return round(sum_ser / (counter-1), 2)

In [323]:
def std_sample(ser):
    mean = mean_sample(ser)
    sigma, counter = 0
    for i in lst(ser):
        counter+=1
        sigma+=(i-mean)**2
    return (sigma/ (counter-1))**0.5

In [324]:
# p value
def Sd_score_sample(ser, result):
    mean = mean_sample(ser)
    std = std_sample(ser)
    return (result-mean)/std

## 2. Importaing databases and calculating the Δs.
### importing the dataframes
notice: most of the cleaning and pre calculation were allready made

In [325]:
df = pd.read_csv("Crime_stats_Ames_City.csv")

In [326]:
df_mean_by_Year= pd.read_csv("Mean_by_Year.csv")

### calculating the crime rate for the number of cases and population to number case for 10,000 people in a year

In [327]:
df =df[["Year","Population","Property crime rate","Burglary rate","Larceny-theft rate","Violent Crime rate","Murder and nonnegligent manslaughter rate","rape rate","Aggravated assault rate"]]

In [328]:
to_div_by_10 = ["Property crime rate","Burglary rate","Larceny-theft rate","Violent Crime rate","Murder and nonnegligent manslaughter rate","rape rate","Aggravated assault rate"]
for i in to_div_by_10:
    df[i] =df[i]/10

In [329]:
df.rename(columns ={"Murder and nonnegligent manslaughter rate":"Murder rate", "rape rate":"Rape rate"}, inplace=True)

### calculating Δ on crime rates

In [330]:
df = df.assign(Year_before = lambda x : x["Year"] - 1)
df_change = df[["Year", "Population","Property crime rate", "Year_before"]]
df_change = df_change.merge(df_change, how = "inner", right_on="Year", left_on="Year_before")

In [331]:
df_change = df_change.assign(Population_change= lambda x: x["Population_x"] - x["Population_y"])
df_change = df_change.assign(Property_crime_rate_change= lambda x: x["Property crime rate_x"] - x["Property crime rate_y"])  
df_change = df_change.drop(["Year_y", "Population_y", "Property crime rate_y", "Year_before_y", "Year_before_x"], axis=1)
df_change.rename(columns ={"Year_x":"Year", "Population_x":"Population", "Property crime rate_x": "Property crime rate"}, inplace=True)

### calculating Δ on sale prices

In [332]:
df_mean_by_Year = df_mean_by_Year[["YrSold", "SalePrice"]]
df_mean_by_Year=df_mean_by_Year.assign(Year_before= lambda x : x["YrSold"] - 1)

In [334]:
df_mean_by_Year = df_mean_by_Year.merge(df_mean_by_Year, how = "inner", right_on="YrSold", left_on="Year_before").assign(avg_price_change=lambda x : x["SalePrice_x"] - x["SalePrice_y"]).drop(['Year_before_x', 'YrSold_y', "Year_before_y", "SalePrice_y" ], axis=1).rename(columns ={"YrSold_x":"Year", "SalePrice_x":"SalePrice"})


### Merging the two data frames toghter and calculating the price change in % 

In [336]:
df_cal = df_2006_2007_change.merge(df_mean_by_Year,how = "inner", right_on="Year", left_on="Year" )

In [337]:
df_cal = df_cal.assign(avg_price_change_pre= lambda x: x["avg_price_change"] * 100 / x["SalePrice"])

In [338]:
df_cal

,Year,Population,Property crime rate,Population_change,Property_crime_rate_change,SalePrice,avg_price_change,avg_price_change_pre
0,2007,51622,310.53,-919,31.70,186063.151976,4042.222489,2.172500
1,2008,55249,296.11,3627,-14.42,177227.602649,-8835.549327,-4.985425
2,2009,57173,282.13,1924,-13.98,179432.103550,2204.500901,1.228599
3,2010,58965,263.89,1792,-18.24,177109.208092,-2322.895458,-1.311561


In [348]:
# statisical data
df_cal.describe()

,Year,Population,Property crime rate,Population_change,Property_crime_rate_change,SalePrice,avg_price_change,avg_price_change_pre
count,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,2008.500000,55752.25000,288.165000,1606.000000,-3.735000,179958.016567,-1227.930349,-0.723972
std,1.290994,3143.91045,19.908232,1879.342616,23.700658,4207.934281,5733.844269,3.199340
min,2007.000000,51622.00000,263.890000,-919.000000,-18.240000,177109.208092,-8835.549327,-4.985425
25%,2007.750000,54342.25000,277.570000,1114.250000,-15.375000,177198.004010,-3951.058925,-2.230027
50%,2008.500000,56211.00000,289.120000,1858.000000,-14.200000,178329.853100,-59.197278,-0.041481
75%,2009.250000,57621.00000,299.715000,2349.750000,-2.560000,181089.865657,2663.931298,1.464574
max,2010.000000,58965.00000,310.530000,3627.000000,31.700000,186063.151976,4042.222489,2.172500


### Calculating the mean, std for the  sample years (2006-2010) 

In [340]:
## sample mean of the property crime rates
mean_property_crime_rate_change = mean_sample(df_cal["Property_crime_rate_change"])
mean_property_crime_rate_change

-4.98

In [341]:
## sample mean of the price change
mean_price_change = mean_sample(df_cal["avg_price_change"])
mean_price_change

-1637.24

In [342]:
## sample mean of the sale price
mean_saleprice = mean_sample(df_cal["SalePrice"])
mean_saleprice

239944.02

### calculting the change in proprety value in the early 21 century in Ames vcity

In [343]:
mean_price_change/mean_saleprice * 100

-0.6823424897190604

### result is 0.68% drop in prices 
#### the result seems to contradict assumtion that there should be an growth in avrage of the prices. but as we know the std of Pope's research (8) we can easly calculate the z score and see that it fall well within the accptable deviation.

In [350]:
z_score = (-0.6823424897190604 - 5.5) /8  

In [351]:
z_score 

-0.7727928112148825

### Hance based on the data we have we have failed to reject our null hypothesis